## Project Proposal
### By: Benjamin Giese & Brady Cassada

#### Identify your dataset

Within our project there will be two datasets, one containing National Hockey League (NHL) team statistics and a second set that has attendance data by team and year. The team data includes each team's overall performance for a single year (wins, goals, penalty minutes, etc.). We plan to combine a subset of each of thes two data sets in order to explore relationships between team performance and attendance.

Team Data: We will be using the 'Teams.csv' from this dataset specifically. [Dataset]('https://www.kaggle.com/open-source-sports/professional-hockey-database#Teams.csv') 

Attendance Data: ESPN has public data of a Team's Home/Away/Overall attendance from 2000-Present day. This isn't already formatted as we will have to collect it all and clean it to our needs for analyzing. [One year of data]('http://www.espn.com/nhl/attendance/_/year/2019')

We'll have to join these two datasets together in some way, this will be done via year and team name. Knowing the 'Teams.csv' data only goes up to 2011, and the ESPN data starts 2000 we will have completed data from the years 2000-2011. 

#### Objectives for analyzing your dataset

Our goal is to find a relationship between and ideally predict attendance based on one or more of the following factors:
* Wins
* Average goals per game
* Average penalty minutes
    
Some additional thoughts:
* The team stats data set includes data for teams that are in leagues other than the NHL. We will only be focusing on NHL teams.
* The first 10 rows of this table are all from 1909, look pretty lame, and are missing a lot of values. Data collection has obviously come a long way in the last 100 years so the more recent data that we will be looking at (2000-2011) is pretty much free of missing values, **this is why we used the tail function instead of head**.
* Due to a lockout, there was no 2004-2005 NHL season.

In [1]:
# First Ten Records of Datset

import pandas as pd
#import matplotlib.pyplot as plt

df = pd.read_csv('Teams.csv')

df.tail(10)

,year,lgID,tmID,franchID,confID,divID,rank,playoff,G,W,...,GA,name,PIM,BenchMinor,PPG,PPC,SHA,PKG,PKC,SHF
1509,2011,NHL,PHI,PHI,EC,AT,3,CSF,82,47,...,232,Philadelphia Flyers,1318.0,18.0,66.0,335.0,9.0,58.0,319.0,6.0
1510,2011,NHL,PHO,PHO,WC,PC,1,CF,82,42,...,204,Phoenix Coyotes,757.0,18.0,34.0,251.0,6.0,36.0,249.0,6.0
1511,2011,NHL,PIT,PIT,EC,AT,2,CQF,82,51,...,221,Pittsburgh Penguins,880.0,12.0,57.0,289.0,10.0,33.0,270.0,11.0
1512,2011,NHL,SJS,SJS,WC,PC,2,CQF,82,43,...,210,San Jose Sharks,785.0,18.0,57.0,270.0,4.0,52.0,225.0,3.0
1513,2011,NHL,STL,STL,WC,CE,1,CSF,82,49,...,165,St. Louis Blues,1062.0,16.0,45.0,270.0,3.0,40.0,282.0,7.0
1514,2011,NHL,TBL,TBL,EC,SE,3,NaN,82,38,...,281,Tampa Bay Lightning,865.0,16.0,41.0,269.0,12.0,59.0,284.0,2.0
1515,2011,NHL,TOR,TOR,EC,NE,4,NaN,82,35,...,264,Toronto Maple Leafs,824.0,16.0,49.0,267.0,6.0,55.0,242.0,5.0
1516,2011,NHL,VAN,VAN,WC,NW,1,CQF,82,51,...,198,Vancouver Canucks,1049.0,10.0,57.0,288.0,4.0,40.0,286.0,7.0
1517,2011,NHL,WAS,WAS,EC,SE,2,CSF,82,42,...,230,Washington Capitals,767.0,16.0,41.0,245.0,10.0,49.0,266.0,3.0
1518,2011,NHL,WPG,WPG,EC,SE,4,NaN,82,37,...,246,Winnipeg Jets,905.0,4.0,45.0,251.0,8.0,58.0,292.0,3.0


#### Team Member: Brady Cassada/Ben Giese